# SetUp

In [1]:
!rm -r Modules
!git clone https://github.com/Trip-Builder/Modules.git
!pip install selenium

Cloning into 'Modules'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 122 (delta 57), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (122/122), 36.29 KiB | 3.30 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [2]:
import warnings
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import numpy as np
import pandas as pd
import Modules.TripBuilder as TB
from multiprocessing import Pool

import urllib
import json
import urllib.parse
import requests
import time
from bs4 import BeautifulSoup

warnings.filterwarnings(action='ignore')

# Multi_Crawler

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
save_path = "/content/drive/MyDrive/트립빌더/TRIFIT"

place = pd.read_csv(save_path+'/jeju_travel_data_231018.csv')

In [5]:
catlst = list(place['category'].unique())
catlst

['쇼핑', '레포츠', '관광지', '음식점', '행사/공연/축제', '숙박', '문화시설']

In [6]:
place_1 = place[place['category'] == '쇼핑']
place_2 = place[place['category'] == '레포츠']
place_3 = place[place['category'] == '관광지']
place_4 = place[place['category'] == '음식점']
place_5 = place[place['category'] == '행사/공연/축제']
place_6 = place[place['category'] == '숙박']
place_7 = place[place['category'] == '문화시설']

In [7]:
place_dataframes = [place_1, place_2, place_3, place_4, place_5, place_6, place_7]

for df in place_dataframes:
    df["startTime"] = [np.nan for i in range(len(df))]
    df["endTime"] = [np.nan for i in range(len(df))]

In [8]:
place = pd.concat(place_dataframes)
place.reset_index(inplace=True,drop=True)

In [ ]:
place = place.iloc[3*len(place)//4:]
place.reset_index(inplace=True,drop=True)

In [9]:
crawler = TB.NaverCrawler()

def get_links(query,num): # 블로그의 게시글 링크들을 가져옵니다.
    data = crawler.get_BlogURL(query,num)
    return data

def get_content(url):
    content,_ = crawler.get_BlogInfo(url)
    if len(content)==0:
      return None
    return content

def Multi_Crawler(query,num):
    pool = Pool(processes=10)
    urls, tot_num = get_links(query,num)
    lst = pool.map(get_content, urls)
    return lst, tot_num

def getTime(sec):
  sec = int(sec)
  h = sec//3600
  m = (sec-h*3600)//60
  s = (sec-h*3600)%60
  return f"{h}h {m}m {s}s"

In [ ]:
# PLA000 Version 추출 -> 후가공
# 일 최대 10만개 / 월 최대 300만개 장소 데이터 수집 가능(poi 이슈로 인해)

crawler = TB.NaverCrawler()
poi_crawler = TB.KakaoCrawler()

name = []; ids = []
cate1 = []; cate2 = []
locX = []; locY = []
address = []; callnum = []
op_time = []; restday = []
contents = []; imgs = []
tag = []; num_txt = [];
spend_avg = []; fee = []
dem_info = []; staytime = []
visit = []; bus_code = []
filter = []; cong = []

s = time.time()

id_num = 0

for i in range(len(place)):
  conts = []; cont_lst = []

  p_info = place["addr"][i].split()
  B_city = p_info[0].strip("특별시").strip("광역시").strip("특별자치도")
  s_city = p_info[1]

  p_info = B_city#" ".join([B_city,s_city])

  place_name = p_info+" "+place["name"][i]
  try:
    conts, tot_num = Multi_Crawler(place_name,40)

    for cont in conts:
      if(cont!=None):
        cont_lst.append(cont)

    position_XY, place_url, cate_1, cate_2, phone_num = poi_crawler.get_Detail(place_name, version=1)

    if len(str(phone_num))==3:
      pass #phone_num = place["telNo"][i]

    if np.isnan(float(position_XY[0])):
      if place['lon'][i]<place['lat'][i]:
        place['lon'][i],place['lat'][i] = place['lat'][i],place['lon'][i]

      position_XY = [place['lon'][i],place['lat'][i]]

    try:
      if np.isnan(cate_1):
        cate_1 = place['category'][i].split(">")[0]
        if cate_1 == "카페/디저트":
          cate_1 = "카페"
        elif cate_1 in ["맛집","음식점"]:
          cate_1 = "음식점"
        cate_2 = place['category'][i].split(">")[-1]
    except:
      pass

    optime = [place["startTime"][i],place["endTime"][i]]


    name.append(place["name"][i])
    ids.append(id_num); id_num+=1
    cate1.append(cate_1)
    cate2.append(cate_2)
    locX.append(position_XY[0])
    locY.append(position_XY[1])
    address.append(place["addr"][i])
    callnum.append(phone_num)
    op_time.append(optime)
    restday.append(np.nan)
    contents.append(cont_lst)
    imgs.append(np.nan)
    tag.append(np.nan)
    num_txt.append(tot_num)
    spend_avg.append(np.nan)
    fee.append(np.nan)
    dem_info.append(np.nan)
    staytime.append(np.nan)
    visit.append(np.nan)
    bus_code.append(np.nan)
    filter.append(np.nan)
    cong.append(np.nan)

    t = time.time()
    times = (t-s)*(len(place)-i-1)/(i+1)
    print(f"\r[{i+1}/{len(place)}]|"+place["name"][i]+f"[{tot_num}] Completed...|[Remaining time:{getTime(times)}]",end="")

  except:
    pass


tot_df = pd.DataFrame({"name":name,"id":ids,"cate1":cate1,"cate2":cate2,"loc_x":locX,"loc_y":locY,"address":address,"callnum":callnum,"time":op_time,"restday":restday,"contents":contents,"imgs":imgs,"tag":tag,"num_txt":num_txt,"spend_avg":spend_avg,"fee":fee,"demographic_info":dem_info,"staytime":staytime,"visit":visit,"business_code":bus_code,"filter":filter,"cong":cong})

data_path = "/content/drive/MyDrive/트립빌더/"
tot_df.to_json(data_path+f"Jeju_social_data_PLA000_4.json")

[113/2477]|뉴월드_마트로 탑동점[50] Completed...|[Remaining time:9h 37m 4s]

Process ForkPoolWorker-594:
Process ForkPoolWorker-572:
Process ForkPoolWorker-601:
Process ForkPoolWorker-605:
Process ForkPoolWorker-602:
Process ForkPoolWorker-606:
Process ForkPoolWorker-580:
Process ForkPoolWorker-603:
Process ForkPoolWorker-604:
Process ForkPoolWorker-608:
Process ForkPoolWorker-575:
Process ForkPoolWorker-578:
Process ForkPoolWorker-571:
Process ForkPoolWorker-609:
Process ForkPoolWorker-610:
Process ForkPoolWorker-574:
Process ForkPoolWorker-576:
Process ForkPoolWorker-573:
Process ForkPoolWorker-579:
Process ForkPoolWorker-577:
Process ForkPoolWorker-607:
Exception ignored in: Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Trace